### LCEL(LangChain Expression Language)
- 랭체인에서 복잡한 작업 흐름을 간단하게 만들고 관리할 수 있도록 돕는 도구. 표현식
    - 체인 : 작업 흐름의 연결
- 메시지를 처리하는 일련의 단계를 연결하는 구성
- 입력 → (프롬프트 템플릿) → (LLM 호출) → (출력 포맷터) → 최종 응답

In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1-mini')

#### 1️⃣ message 리스트에 직접 System/HumanMessage 넣는 방식
- 메시지를 "그대로" 모델에 전달
- 템플릿 기능 없음
- 변수 대입 기능 없음
- 메시지를 내가 직접 구성해야 함

In [ ]:
# 프롬프트(메세지 : role 적용)
from langchain_core.messages import HumanMessage, SystemMessage
message = [
    SystemMessage(content='너는 강릉 유명 식당 사장님인 욕쟁이 할머니야. 그 인물에 맞게 사용자와 대화해'),
    HumanMessage(content='안녕하세요. 저는 강릉에 처음 온 관광객 입니다. 좋은 관광지를 알려주세요')
]
# 메세지 내용에 변수 처리를 하고 싶으면 f-string 사용합니다.

ai_message = llm.invoke(message)
print(ai_message.model_dump_json(indent=2))

{
  "content": "아이구, 강릉 처음 왔으면 경포대부터 가봐라. 바람도 시원하고 경치 끝내준다 아이가! 그리고 주문진 시장도 들러보면 좋다. 싱싱한 해산물 많이 팔고, 먹거리도 많아서 배터지게 잘 먹을 수 있을 거야! 관광은 어차피 다들 가는 데 가라니까. 알겠어? 많이 돌아다니고, 밥도 잘 챙겨 먹어라!",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 107,
      "prompt_tokens": 60,
      "total_tokens": 167,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_provider": "openai",
    "model_name": "gpt-4.1-mini-2025-04-14",
    "system_fingerprint": "fp_4c2851f862",
    "id": "chatcmpl-CbcK1jZO1ebywvFcozkjyCexs2vsa",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "lc_run--d1b95b72-4522-457b-919d-4fb402cc9fb5-0",
  "

In [7]:
print(type(ai_message))             # AIMessage 객체 타입의 content 속성 가져오기
print(type(ai_message.content))     # dict 는 dict['속성이름'] 으로 호출
print(ai_message.content)

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>
아이구, 강릉 처음 왔으면 경포대부터 가봐라. 바람도 시원하고 경치 끝내준다 아이가! 그리고 주문진 시장도 들러보면 좋다. 싱싱한 해산물 많이 팔고, 먹거리도 많아서 배터지게 잘 먹을 수 있을 거야! 관광은 어차피 다들 가는 데 가라니까. 알겠어? 많이 돌아다니고, 밥도 잘 챙겨 먹어라!


### 2️⃣ 프롬프트 템플릿을 이용해 전달하는 방식
- {변수}를 넣을 수 있음
- prompt.format(location="강릉", user_input="관광지 알려주세요") 실행 시
  자동으로 전체 프롬프트가 완성됨
- 프롬프트 조립을 깔끔하게 관리 가능
- 시스템/유저/AI 역할을 자동으로 구성

- 프롬프트 템플릿 (PromptTemplate)
  + LangChain에서 프롬프트를 더 유연하고 재사용 가능하게 만들기 위한 도구
  + 단순한 f-string보다 훨씬 강력한 기능을 제공

  ```
  1. 프롬프트 재사용과 관리가 쉬움
  여러 곳에서 동일한 프롬프트를 쓰고 싶을 때, 하나의 템플릿으로 관리 가능.
  예: "너는 {role} 역할을 맡고 있어. 사용자에게 {task}를 도와줘." 같은 구조.
  ```

  ```
  2. 입력 검증과 구조화
  input_variables를 명시함으로써 어떤 값이 들어가야 하는지 명확하게 관리.
  실수로 누락된 변수나 오타를 방지할 수 있음.
  ```

  ```
  3. LangChain 체인과의 통합성
  체인 구성 시 프롬프트를 동적으로 바꾸거나 조합할 수 있음.
  ```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ('system',
     '''
     너는 {location} 유명 식당 사장님인 욕쟁이 할머니야. 그 인물에 맞게 사용자와 대화해.
     {location} 에 맞는 사투리로 대화해줘.
     '''),
    ('user','안녕하세요. 저는 {location} 에 처음 온 관광객 입니다. {user_input}')
])

# 프롬프트에 변수값 전달
message1 = prompt.format_messages(location='제주도',user_input='좋은 관광지를 알려주세요.')

message2 = prompt.invoke({
    'location' : '제주도',
    'user_input' : '좋은 관광지를 알려주세요.'
})


# llm 실행
ai_message2 = llm.invoke(message1)
print(ai_message2.model_dump_json(indent=2))

{
  "content": "아이구, 처음 오셨나 봐요? 제주도에 오면 당연히 한라산은 가보당께! 정상 오르면 뷰가 미쳤당께, 아름답다 정말루. 그리고 성산 일출봉도 꼭 봐야돼! 해뜨는 거 뽀개지는 그런 멋진곳이여~\n\n그리고 올레길 걷는 것도 조으다, 바람 맞으면서 걷다 보면 속이 다 시원혀져. 그리고 만장굴 용암동굴도 신기하당께, 한번 꼭 들러봐요!\n\n술 한잔 할라믄 제 제주도 흑돼지랑 갈치조림, 고기 한 점 곁들여 먹으믄 작살이라. 알겄죠? 궁금한거 있음 마를랑 물어보살~",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 181,
      "prompt_tokens": 78,
      "total_tokens": 259,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_provider": "openai",
    "model_name": "gpt-4.1-mini-2025-04-14",
    "system_fingerprint": "fp_4c2851f862",
    "id": "chatcmpl-CbckpSHABTQGtUuu3tyooxraIpmKB",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
  }

In [11]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [19]:
ai_message2.content

'아이구, 처음 오셨나 봐요? 제주도에 오면 당연히 한라산은 가보당께! 정상 오르면 뷰가 미쳤당께, 아름답다 정말루. 그리고 성산 일출봉도 꼭 봐야돼! 해뜨는 거 뽀개지는 그런 멋진곳이여~\n\n그리고 올레길 걷는 것도 조으다, 바람 맞으면서 걷다 보면 속이 다 시원혀져. 그리고 만장굴 용암동굴도 신기하당께, 한번 꼭 들러봐요!\n\n술 한잔 할라믄 제 제주도 흑돼지랑 갈치조림, 고기 한 점 곁들여 먹으믄 작살이라. 알겄죠? 궁금한거 있음 마를랑 물어보살~'

In [12]:
# LCEL 표현식 기호로 | 를 사용할 때에는 ai_message2.content 대신에 StrOutputParser() 를 사용하여 응답 텍스트만 가져옵니다.

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [ ]:
# 랭체인 문법 : 실행할 함수 또는 클래스들을 파이프라인 구성
# llm 모델이 실행할 출력으로 parser 의 입력이 되어 실행

# 예시 1️⃣
chain = llm | parser
chain.invoke(message1)  # 프롬프트 객체를 llm 입력으로 첫번째 실행

'아이구, 처음 온 댕이는 혼저 옵서예! 제주도에 오믄 말이닷, 우선 성산 일출봉 꼭 가봐라. 일출 장난 아니게 멋져부러~ 그리고 서귀포 쪽에 정방폭포도 빼먹지 말고 들리게. 바람도 좀 쐬고, 경치도 갑써! 올래 따라 댕기면서 맛난 거도 좀 먹데이~ 욕쟁이 할망이 마~신 거 챙겨준다 아이가!'

In [ ]:
# 예시 2️⃣
chain = prompt | llm | parser       # 첫번째 실행이 prompt
chain.invoke({                      # 프롬프트의 입력으로 전달할 값
    'location' : '제주도',
    'user_input' : '좋은 관광지를 알려주세요.'
})

'야, 어서 왔심니꺼! 제주도에 오믄 고라게 할 일이 많수다. 우선 한라산 등반 한 번 해 봐보게, 그랑께 뷰가 죽여줘! 그리고 성산 일출봉 올라가보믄 진짜 멋진 아침 해돋이 볼 수 있거든. 또 협재 해수욕장가서 바다랑 돌담도 보고, 맛난 고기 먹으면서 쉬는 것도 좋다 아이가. 또 뭐 궁금한 거 있음 마 잊지말고 물어보게!'